# Time Complexity & Scaling Analysis

In this notebook, we analyze how the runtime of different option pricing methods scales with increasing computational resolution. The focus is on:

- Crank-Nicolson PDE method
- Monte Carlo simulation
- Binomial Tree method

We benchmark runtimes against increasing resolution parameters and assess the practical trade-offs between speed and accuracy.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os, sys

sys.path.append(os.path.abspath("../../"))

from pricing.bsm import black_scholes_price
from pricing.monte_carlo import MonteCarloPricer
from pricing.pde import CrankNicolsonSolver
from pricing.binomial_tree import BinomialVanillaPricer

In [ ]:
# Parameters

S0 = 100
K = 100
T = 1.0
r = 0.05
sigma = 0.2
is_call = True

bsm_ref = black_scholes_price(S0, K, T, r, sigma, option_type="call")
print(f"BSM Reference Price: {bsm_ref:.4f}")

In [ ]:
# Measure PDE Runtimes

pde_steps = np.arange(20, 220, 20)
pde_times = []

for N in pde_steps:
    start = time.time()
    solver = CrankNicolsonSolver(S_max=200, K=K, T=T, r=r, sigma=sigma, M=100, N=N, is_call=is_call)
    solver.solve()
    pde_times.append(time.time() - start)

In [ ]:
# Measure MC Runtimes

mc_paths = [1000, 2000, 4000, 8000, 16000]
mc_times = []

for n_paths in mc_paths:
    start = time.time()
    pricer = MonteCarloPricer(S0, K, T, r, sigma, n_paths=n_paths, option_type='call')
    pricer.simulate_payoffs()
    mc_times.append(time.time() - start)

In [ ]:
# Measure Binomial Runtimes

binomial_steps = np.arange(50, 1050, 100)
binomial_times = []

for N in binomial_steps:
    start = time.time()
    pricer = BinomialVanillaPricer(S0, K, T, r, sigma, N, is_call)
    binomial_times.append(time.time() - start)

In [ ]:
# Plot Runtime Scaling

plt.figure(figsize=(10, 6))
plt.plot(pde_steps, pde_times, label="PDE (Crank-Nicolson)", marker='s')
plt.plot(mc_paths, mc_times, label="Monte Carlo", marker='o')
plt.plot(binomial_steps, binomial_times, label="Binomial Tree", marker='^')

plt.xlabel("Resolution (Grid steps / Paths)")
plt.ylabel("Runtime (seconds)")
plt.title("Time Complexity & Scaling of Pricing Methods")
plt.legend()
plt.grid(True)
plt.show()

### Observations

- **PDE (Crank-Nicolson)** shows nearly linear scaling with the number of time steps due to matrix operations.
- **Monte Carlo** runtime increases linearly with the number of paths but has relatively low slope (highly parallelizable).
- **Binomial Tree** scales roughly quadratically with number of steps due to nested loops.

PDE is fast for moderate grids, MC is best for high-dimensional options, and Binomial is easy to implement but computationally expensive for high steps.

In [ ]:
pde_errors, pde_times_acc = [], []
for N in pde_steps:
    start = time.time()
    solver = CrankNicolsonSolver(S_max=200, K=K, T=T, r=r, sigma=sigma, M=100, N=N, is_call=is_call)
    price_vec, grid = solver.solve()
    price = np.interp(S0, grid, price_vec)
    pde_times_acc.append(time.time() - start)
    pde_errors.append(abs(price - bsm_ref))

mc_errors, mc_times_acc = [], []
for n_paths in mc_paths:
    start = time.time()
    pricer = MonteCarloPricer(S0, K, T, r, sigma, n_paths=n_paths, option_type='call')
    pricer.simulate_payoffs()
    mc_times_acc.append(time.time() - start)
    mc_errors.append(abs(price - bsm_ref))

binomial_errors, binomial_times_acc = [], []
for N in binomial_steps:
    start = time.time()
    price = BinomialVanillaPricer(S0, K, T, r, sigma, N, is_call).price()
    binomial_times_acc.append(time.time() - start)
    binomial_errors.append(abs(price - bsm_ref))

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(pde_times_acc, pde_errors, label="PDE", marker='s')
plt.plot(mc_times_acc, mc_errors, label="Monte Carlo", marker='o')
plt.plot(binomial_times_acc, binomial_errors, label="Binomial", marker='^')

plt.xlabel("Runtime (seconds)")
plt.ylabel("Absolute Error vs BSM")
plt.title("Runtime vs Accuracy Trade-off")
plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

### Runtime vs Accuracy Summary

- **PDE Method** achieves low error quickly, making it efficient for vanilla options.
- **Monte Carlo** improves accuracy slowly over time, but it's scalable and flexible for exotic options.
- **Binomial Tree** shows higher initial error and runtime grows quickly — less efficient for high accuracy.

This comparison highlights that **PDE methods dominate vanilla option pricing**, but **Monte Carlo remains competitive** for flexibility, especially when payoff structure becomes complex.